In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import precision_score, accuracy_score
import numpy as np

# Define WideResNet model (wrn-28-10 as an example)
class BasicBlock(nn.Module):
    def __init__(self, in_planes, out_planes, stride, drop_rate):
        super(BasicBlock, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.conv1 = nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_planes)
        self.conv2 = nn.Conv2d(out_planes, out_planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.drop_rate = drop_rate
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != out_planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)
            )

    def forward(self, x):
        out = F.relu(self.bn1(x))  # Removed inplace=True
        out = self.conv1(out)
        out = F.relu(self.bn2(out))  # Removed inplace=True
        if self.drop_rate > 0:
            out = F.dropout(out, p=self.drop_rate)
        out = self.conv2(out)
        out += self.shortcut(x)
        return out


class WideResNet(nn.Module):
    def __init__(self, depth, num_classes, widen_factor=10, drop_rate=0.3):
        super(WideResNet, self).__init__()
        self.in_planes = 16
        assert ((depth - 4) % 6 == 0), 'WideResNet depth should be 6n+4'
        n = (depth - 4) // 6
        k = widen_factor

        self.conv1 = nn.Conv2d(3, self.in_planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.layer1 = self._wide_layer(BasicBlock, 16 * k, n, stride=1, drop_rate=drop_rate)
        self.layer2 = self._wide_layer(BasicBlock, 32 * k, n, stride=2, drop_rate=drop_rate)
        self.layer3 = self._wide_layer(BasicBlock, 64 * k, n, stride=2, drop_rate=drop_rate)
        self.bn1 = nn.BatchNorm2d(64 * k)
        self.linear = nn.Linear(64 * k, num_classes)

    def _wide_layer(self, block, planes, num_blocks, stride, drop_rate):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride, drop_rate))
            self.in_planes = planes
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.relu(self.bn1(out))  # Removed inplace=True
        out = F.avg_pool2d(out, 8)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

# Focal Loss implementation
class FocalLoss(nn.Module):
    def __init__(self, gamma=2, alpha=0.25, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        self.reduction = reduction

    def forward(self, inputs, targets):
        BCE_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1 - pt) ** self.gamma * BCE_loss
        if self.reduction == 'mean':
            return F_loss.mean()
        else:
            return F_loss.sum()

# Data preparation
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)),
])

train_dataset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)

train_size = 40000
val_size = 10000
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=4)

# Model, loss function, optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = WideResNet(depth=28, num_classes=100).to(device)
criterion = FocalLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

# Helper functions to calculate accuracy and precision
def calculate_metrics(output, target):
    pred = torch.argmax(output, dim=1)
    accuracy = accuracy_score(target.cpu(), pred.cpu())
    precision = precision_score(target.cpu(), pred.cpu(), average='macro')
    return accuracy, precision

def evaluate(loader):
    model.eval()
    correct = 0
    total = 0
    all_targets = []
    all_preds = []
    with torch.no_grad():
        for data in loader:
            inputs, targets = data
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            all_targets.append(targets)
            all_preds.append(outputs)
    all_targets = torch.cat(all_targets)
    all_preds = torch.cat(all_preds)
    accuracy, precision = calculate_metrics(all_preds, all_targets)
    return accuracy, precision

# Training loop
num_epochs = 200
for epoch in range(num_epochs):
    model.train()
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

    scheduler.step()

    if (epoch + 1) % 10 == 0:
        train_accuracy, train_precision = evaluate(train_loader)
        val_accuracy, val_precision = evaluate(val_loader)
        test_accuracy, test_precision = evaluate(test_loader)
        print(f'Epoch [{epoch+1}/{num_epochs}]')
        print(f'Train Accuracy: {train_accuracy:.4f}, Train Precision: {train_precision:.4f}')
        print(f'Val Accuracy: {val_accuracy:.4f}, Val Precision: {val_precision:.4f}')
        print(f'Test Accuracy: {test_accuracy:.4f}, Test Precision: {test_precision:.4f}')


100%|██████████| 169001437/169001437 [00:02<00:00, 58545191.78it/s]


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified
Epoch [10/200]
Train Accuracy: 0.5022, Train Precision: 0.5625
Val Accuracy: 0.4551, Val Precision: 0.5193
Test Accuracy: 0.4706, Test Precision: 0.5318
Epoch [20/200]
Train Accuracy: 0.5535, Train Precision: 0.6286
Val Accuracy: 0.4914, Val Precision: 0.5657
Test Accuracy: 0.4987, Test Precision: 0.5777
Epoch [30/200]
Train Accuracy: 0.6003, Train Precision: 0.6696
Val Accuracy: 0.5191, Val Precision: 0.5996
Test Accuracy: 0.5287, Test Precision: 0.6049
Epoch [40/200]
Train Accuracy: 0.6446, Train Precision: 0.7127
Val Accuracy: 0.5385, Val Precision: 0.6121
Test Accuracy: 0.5638, Test Precision: 0.6370
Epoch [50/200]
Train Accuracy: 0.6621, Train Precision: 0.7300
Val Accuracy: 0.5521, Val Precision: 0.6288
Test Accuracy: 0.5639, Test Precision: 0.6433
Epoch [60/200]
Train Accuracy: 0.6999, Train Precision: 0.7379
Val Accuracy: 0.5797, Val Precision: 0.6240
Test Accuracy: 0.5806, Test Precision: